In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/Priyanka/Desktop/data_analyst project/dataset-3.csv")

In [3]:
df.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


# Question 1: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A)

In [13]:
# import pandas as pd

# def calculate_distance_matrix(input_csv):
#     # Read the CSV file into a DataFrame
#     df = pd.read_csv(input_csv, index_col='id_start')

#     # Create a dictionary to store distances between toll locations
#     distances = {}

#     # Iterate through the DataFrame to populate the distances dictionary
#     for source in df.index:
#         for destination in df.index:
#             if source != destination:
#                 # Check if the distance is already calculated
#                 if (source, destination) not in distances:
#                     # Bidirectional distance between toll locations
#                     distance_forward = df.loc[source, destination]
#                     distance_backward = df.loc[destination, source]

#                     # Calculate cumulative distance
#                     total_distance = distance_forward + distance_backward

#                     # Update distances dictionary
#                     distances[(source, destination)] = total_distance

#     # Create a DataFrame from the distances dictionary
#     distance_matrix = pd.DataFrame(index=df.index, columns=df.index)

#     # Populate the distance matrix
#     for source in df.index:
#         for destination in df.index:
#             if source == destination:
#                 distance_matrix.loc[source, destination] = 0
#             else:
#                 distance_matrix.loc[source, destination] = distances.get((source, destination), distances.get((destination, source), float('inf')))

#     return distance_matrix

# # Example usage
# input_csv = 'dataset-3.csv'
# resulting_matrix = calculate_distance_matrix(input_csv)
# resulting_matrix


In [5]:
import pandas as pd

 # Read the CSV file into a DataFrame
def calculate_distance_matrix(input_csv):
    df = pd.read_csv(input_csv)

# Create a dictionary to store distances between toll locations
    distances = {}

# Iterate through the DataFrame to populate the distances dictionary
    for index, row in df.iterrows():
        source = row['id_start']
        destination = row['id_end']
        distance_forward = row['distance']

# Check if the distance is already calculated
        if (source, destination) not in distances:            
            distance_backward = df[(df['id_start'] == destination) & (df['id_end'] == source)]['distance'].values # Bidirectional distance between toll locations
            if len(distance_backward) > 0:
                distance_backward = distance_backward[0]
                total_distance = distance_forward + distance_backward      # Calculate cumulative distance
                distances[(source, destination)] = total_distance          # Calculate cumulative distance

 # Create a DataFrame from the distances dictionary
    distance_matrix = pd.DataFrame(index=df['id_start'].unique(), columns=df['id_start'].unique())

 # Populate the distance matrix
    for source in distance_matrix.index:
        for destination in distance_matrix.columns:
            if source == destination:
                distance_matrix.at[source, destination] = 0
            else:
                distance_matrix.at[source, destination] = distances.get((source, destination), distances.get((destination, source), float('inf')))

    return distance_matrix

# Example usage
input_csv = 'dataset-3.csv'
resulting_matrix = calculate_distance_matrix(input_csv)
print(resulting_matrix)


        1001400 1001402 1001404 1001406 1001408 1001410 1001412 1001414  \
1001400       0     inf     inf     inf     inf     inf     inf     inf   
1001402     inf       0     inf     inf     inf     inf     inf     inf   
1001404     inf     inf       0     inf     inf     inf     inf     inf   
1001406     inf     inf     inf       0     inf     inf     inf     inf   
1001408     inf     inf     inf     inf       0     inf     inf     inf   
1001410     inf     inf     inf     inf     inf       0     inf     inf   
1001412     inf     inf     inf     inf     inf     inf       0     inf   
1001414     inf     inf     inf     inf     inf     inf     inf       0   
1001416     inf     inf     inf     inf     inf     inf     inf     inf   
1001418     inf     inf     inf     inf     inf     inf     inf     inf   
1001420     inf     inf     inf     inf     inf     inf     inf     inf   
1001422     inf     inf     inf     inf     inf     inf     inf     inf   
1001424     inf     inf  

# Question 2: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [6]:
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Create an empty list to store unrolled distances
    unrolled_distances = []

    # Iterate through the distance matrix to generate combinations
    for id_start in distance_matrix.index:
        for id_end in distance_matrix.columns:
            if id_start != id_end:
                distance = distance_matrix.at[id_start, id_end]
                unrolled_distances.append([id_start, id_end, distance])

    # Create a DataFrame from the unrolled distances list
    unrolled_df = pd.DataFrame(unrolled_distances, columns=['id_start', 'id_end', 'distance'])

    return unrolled_df

# Example usage with resulting_matrix from Question 1
resulting_matrix = calculate_distance_matrix('dataset-3.csv')
unrolled_result = unroll_distance_matrix(resulting_matrix)
print(unrolled_result)


      id_start   id_end  distance
0      1001400  1001402       inf
1      1001400  1001404       inf
2      1001400  1001406       inf
3      1001400  1001408       inf
4      1001400  1001410       inf
...        ...      ...       ...
1635   1001470  1001461       inf
1636   1001470  1001462       inf
1637   1001470  1001464       inf
1638   1001470  1001466       inf
1639   1001470  1001468       inf

[1640 rows x 3 columns]


# Question 3: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [9]:
import pandas as pd

def find_ids_within_ten_percentage_threshold(df, reference_value):
    # Filter rows for the given reference value
    reference_rows = df[df['id_start'] == reference_value]

    # Check if the reference value exists in the DataFrame
    if reference_rows.empty:
        raise ValueError(f"Reference value {reference_value} not found in the DataFrame.")

    # Calculate the average distance for the reference value
    reference_avg_distance = reference_rows['distance'].mean()

    # Check if the average distance is not NaN and not zero
    if pd.isna(reference_avg_distance) or reference_avg_distance == 0:
        raise ValueError(f"Invalid average distance for reference value {reference_value}.")

    # Calculate the threshold range (10% of the average distance)
    threshold = 0.1 * reference_avg_distance

    # Filter rows within the 10% threshold
    within_threshold_rows = df[(df['distance'] >= reference_avg_distance - threshold) &
                               (df['distance'] <= reference_avg_distance + threshold)]

    # Get unique values from the 'id_start' column and sort them
    within_threshold_ids = sorted(within_threshold_rows['id_start'].unique())

    return within_threshold_ids

# Example usage with unrolled_result from Question 2 and a reference value (e.g., 1001420)
reference_value = 1001420
ids_within_threshold = find_ids_within_ten_percentage_threshold(unrolled_result, reference_value)
print(ids_within_threshold)


[]


C:\Users\Priyanka\AppData\Local\Temp\ipykernel_19400\258432054.py:22: RuntimeWarning: invalid value encountered in double_scalars
  within_threshold_rows = df[(df['distance'] >= reference_avg_distance - threshold) &


# Question 4: Calculate Toll Rate
Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

* 0.8 for moto
* 1.2 for car
* 1.5 for rv
* 2.2 for bus
* 3.6 for truck

In [10]:
import pandas as pd

def calculate_toll_rate(df):
    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate in rate_coefficients.items():
        df[vehicle_type] = df['distance'] * rate

    return df

# Example usage with unrolled_result from Question 2
result_with_toll_rates = calculate_toll_rate(unrolled_result)
print(result_with_toll_rates)


      id_start   id_end  distance  moto  car   rv  bus  truck
0      1001400  1001402       inf   inf  inf  inf  inf    inf
1      1001400  1001404       inf   inf  inf  inf  inf    inf
2      1001400  1001406       inf   inf  inf  inf  inf    inf
3      1001400  1001408       inf   inf  inf  inf  inf    inf
4      1001400  1001410       inf   inf  inf  inf  inf    inf
...        ...      ...       ...   ...  ...  ...  ...    ...
1635   1001470  1001461       inf   inf  inf  inf  inf    inf
1636   1001470  1001462       inf   inf  inf  inf  inf    inf
1637   1001470  1001464       inf   inf  inf  inf  inf    inf
1638   1001470  1001466       inf   inf  inf  inf  inf    inf
1639   1001470  1001468       inf   inf  inf  inf  inf    inf

[1640 rows x 8 columns]


# Question 5: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

* start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
* start_time and end_time must be of type datetime.time() with the values from time range given below.

Modify the values of vehicle columns according to the following time ranges:
Weekdays (Monday - Friday):

* From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
* From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
* From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
* Weekends (Saturday and Sunday):

+ Apply a constant discount factor of 0.7 for all times.

For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).